<a href="https://colab.research.google.com/github/hernita12/hernita_tugas-15_dasar-algoritma-dan-pemrograman/blob/main/bakery_hernita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================
# Program Aplikasi Toko Roti
# Menggunakan Python dan Pandas
# Fitur: input produk, pembelian, simpan CSV
# ==========================

import pandas as pd      # Untuk manipulasi data tabel
import os                # Untuk memeriksa keberadaan file
import json              # Untuk menyimpan keranjang belanja dalam format string JSON

# --------------------------
# Pengaturan nama file CSV
# --------------------------
produk_file = 'produk_roti.csv'         # File data produk
riwayat_file = 'riwayat.csv'            # File data riwayat pembelian

# --------------------------
# Fungsi: Memuat dan menyimpan data
# --------------------------
def load_data():
    """
    Membaca data dari file CSV jika ada,
    jika tidak, membuat DataFrame kosong
    """
    if os.path.exists(produk_file):
        produk = pd.read_csv(produk_file)
    else:
        produk = pd.DataFrame(columns=["Nama", "Harga", "Stok"])

    if os.path.exists(riwayat_file):
        riwayat = pd.read_csv(riwayat_file)
    else:
        riwayat = pd.DataFrame(columns=["Nama Pembeli", "Total Belanja", "Detail"])

    return produk, riwayat

def save_data(produk, riwayat):
    """
    Menyimpan data produk dan riwayat transaksi ke file CSV
    """
    produk.to_csv(produk_file, index=False)
    riwayat.to_csv(riwayat_file, index=False)

# --------------------------
# Fungsi untuk menu penjual
# --------------------------
def tambah_produk(produk):
    """
    Menambahkan data roti baru ke daftar produk
    """
    while True:
        nama = input("Nama roti: ")
        harga = int(input("Harga (Rp): "))
        stok = int(input("Jumlah stok: "))
        produk.loc[len(produk)] = [nama, harga, stok]

        lagi = input("Tambah produk lagi? (y/n): ").lower()
        if lagi != 'y':
            break
    return produk

def lihat_stok(produk):
    """
    Menampilkan seluruh stok roti
    """
    if produk.empty:
        print("Stok kosong.")
    else:
        print("\nDaftar Stok Roti:")
        print(produk)

def lihat_pemasukan(riwayat):
    """
    Menghitung dan menampilkan total pemasukan dari transaksi
    """
    if riwayat.empty:
        print("Belum ada transaksi.")
    else:
        total = riwayat["Total Belanja"].sum()
        print(f"Total pemasukan: Rp{total}")

def lihat_riwayat(riwayat):
    """
    Menampilkan seluruh riwayat pembelian dari pelanggan
    """
    if riwayat.empty:
        print("Belum ada transaksi.")
        return

    print("\nRiwayat Pembelian:")
    for i, row in riwayat.iterrows():
        print(f"{i+1}. {row['Nama Pembeli']} - Total: Rp{row['Total Belanja']}")
        try:
            detail = json.loads(row["Detail"])
            for item in detail:
                print(f"   - {item['Jumlah']} x {item['Nama']} = Rp{item['Subtotal']}")
        except:
            print("   (Data transaksi tidak terbaca)")

def menu_penjual(produk, riwayat):
    """
    Menu utama untuk penjual: tambah produk, lihat stok, pemasukan, dan riwayat
    """
    while True:
        print("""
=== Menu Penjual ===
1. Tambah Produk
2. Lihat Stok
3. Lihat Pemasukan
4. Lihat Riwayat Pembelian
5. Kembali ke Menu Utama
""")
        pilihan = input("Pilih (1-5): ")

        if pilihan == '1':
            produk = tambah_produk(produk)
        elif pilihan == '2':
            lihat_stok(produk)
        elif pilihan == '3':
            lihat_pemasukan(riwayat)
        elif pilihan == '4':
            lihat_riwayat(riwayat)
        elif pilihan == '5':
            break
        else:
            print("Pilihan tidak valid.")
    return produk

# --------------------------
# Fungsi untuk menu pembeli
# --------------------------
def menu_pembeli(produk, riwayat):
    """
    Menu pembeli: memilih dan membeli roti dari stok yang tersedia
    """
    if produk.empty:
        print("Belum ada roti tersedia.")
        return produk, riwayat

    nama = input("Masukkan nama Anda: ")
    keranjang = []
    total = 0

    while True:
        print("\nStok Roti:")
        print(produk)

        nama_roti = input("Nama roti yang ingin dibeli: ")
        jumlah = int(input("Jumlah: "))
        ditemukan = False

        for i, row in produk.iterrows():
            if row["Nama"].lower() == nama_roti.lower():
                ditemukan = True
                if row["Stok"] >= jumlah:
                    subtotal = row["Harga"] * jumlah
                    keranjang.append({
                        "Nama": row["Nama"],
                        "Jumlah": jumlah,
                        "Subtotal": subtotal
                    })
                    total += subtotal
                    produk.at[i, "Stok"] -= jumlah
                    print(f"Ditambahkan: {jumlah} x {row['Nama']} = Rp{subtotal}")
                else:
                    print("Stok tidak cukup.")
                break

        if not ditemukan:
            print("Roti tidak ditemukan.")

        lanjut = input("Beli roti lain? (y/n): ").lower()
        if lanjut != 'y':
            break

    if keranjang:
        print("\nRingkasan Belanja:")
        for item in keranjang:
            print(f"- {item['Jumlah']} x {item['Nama']} = Rp{item['Subtotal']}")
        print(f"Total belanja: Rp{total}")
        riwayat.loc[len(riwayat)] = [nama, total, json.dumps(keranjang)]
        print("Terima kasih atas pembelian Anda.")
    else:
        print("Belum ada pembelian.")

    return produk, riwayat

# --------------------------
# Program Utama
# --------------------------
produk, riwayat = load_data()

while True:
    print("""
====== BAKERY ======
1. Login sebagai Pembeli
2. Login sebagai Penjual
3. Simpan dan Keluar
""")
    menu = input("Pilih (1-3): ")

    if menu == '1':
        produk, riwayat = menu_pembeli(produk, riwayat)
    elif menu == '2':
        produk = menu_penjual(produk, riwayat)
    elif menu == '3':
        save_data(produk, riwayat)
        print("Data telah disimpan. Sampai jumpa.")
        break
    else:
        print("Pilihan tidak valid.")



====== BAKERY ======
1. Login sebagai Pembeli
2. Login sebagai Penjual
3. Simpan dan Keluar

Belum ada roti tersedia.

====== BAKERY ======
1. Login sebagai Pembeli
2. Login sebagai Penjual
3. Simpan dan Keluar


=== Menu Penjual ===
1. Tambah Produk
2. Lihat Stok
3. Lihat Pemasukan
4. Lihat Riwayat Pembelian
5. Kembali ke Menu Utama


=== Menu Penjual ===
1. Tambah Produk
2. Lihat Stok
3. Lihat Pemasukan
4. Lihat Riwayat Pembelian
5. Kembali ke Menu Utama


Daftar Stok Roti:
         Nama  Harga  Stok
0   CROISSANT    300    30
1  CROMBOLONI    200    40

=== Menu Penjual ===
1. Tambah Produk
2. Lihat Stok
3. Lihat Pemasukan
4. Lihat Riwayat Pembelian
5. Kembali ke Menu Utama

Belum ada transaksi.

=== Menu Penjual ===
1. Tambah Produk
2. Lihat Stok
3. Lihat Pemasukan
4. Lihat Riwayat Pembelian
5. Kembali ke Menu Utama

Belum ada transaksi.

=== Menu Penjual ===
1. Tambah Produk
2. Lihat Stok
3. Lihat Pemasukan
4. Lihat Riwayat Pembelian
5. Kembali ke Menu Utama

